In [ ]:
import pandas as pd

In [ ]:
"""
# If importing data from drive

from google.colab import drive
drive.mount('/content/drive')


df = pd.read_csv('drive/MyDrive/CODE/Machine Learning/clustering/raw_data/Clustering_ProductPrices.csv')

"""

In [ ]:
df = pd.read_csv('raw_data/Clustering_ProductPrices.csv')

In [ ]:
#sanity to check if there is one row per date

df.groupby(['ASIN','MARKETPLACE','BRAND_NAME','DATE']).size().max()

1

In [ ]:
df.dtypes

ASIN                   object
MARKETPLACE            object
REGION                 object
BRAND_NAME             object
DATE                   object
UNITS_SOLD_TOTAL      float64
ORGANIC_UNITS_SOLD    float64
SALE_PRICE_EUR        float64
dtype: object

In [ ]:
df.shape

(649283, 8)

In [ ]:
df.dropna().shape

(82547, 8)

In [ ]:
df.isna().sum()

ASIN                     460
MARKETPLACE                0
REGION                     0
BRAND_NAME                 0
DATE                       0
UNITS_SOLD_TOTAL      496852
ORGANIC_UNITS_SOLD    496852
SALE_PRICE_EUR        509474
dtype: int64

In [ ]:
df = df.dropna()


---
Correlation

In [ ]:
import numpy as np
from scipy.stats import pearsonr

corr = lambda x,y: pearsonr(np.array(x), np.array(y))[0]

#on ASIN/MARKET/BRAND level aggregate units sold and sales price to get list of values and count of values per list.
gr_data = df.sort_values('DATE').groupby(['ASIN','MARKETPLACE','BRAND_NAME']).agg({"ORGANIC_UNITS_SOLD":[lambda x: list(x),lambda x: len(list(x))],
                                                                                "SALE_PRICE_EUR":[lambda x: list(x),lambda x: len(list(x))]}).reset_index()


gr_data.columns = ['ASIN','MARKETPLACE','BRAND_NAME','ORGANIC_UNITS_SOLD_list','ORGANIC_UNITS_SOLD_count',
                 'SALE_PRICE_EUR_list','SALE_PRICE_EUR_count']

gr_data_refined = gr_data[gr_data['ORGANIC_UNITS_SOLD_count'] > 2] #get entries with more than 2 units sold
gr_data_refined['correlation_coeff'] = gr_data_refined.apply(lambda x: corr(x['ORGANIC_UNITS_SOLD_list'],x['SALE_PRICE_EUR_list']),axis=1).fillna(0) #get corr coef

subset = gr_data_refined[['ASIN','MARKETPLACE','BRAND_NAME','correlation_coeff']]
subset.to_csv('preprocessed_data/CorrelationCoefficient.csv',index=False)


# If using Drive
#subset.to_csv('drive/MyDrive/CODE/Machine Learning/clustering/preprocessed_data/CorrelationCoefficient.csv',index=False)


In [ ]:
#sanity to check if there is one row per date

avg_df = df.dropna().groupby(['ASIN','MARKETPLACE','BRAND_NAME']).agg({"ORGANIC_UNITS_SOLD":'mean',
                                                     "SALE_PRICE_EUR":'mean'}).reset_index().sort_values('SALE_PRICE_EUR')


avg_df.to_csv('preprocessed_data/AvgPriceAndSalesVolume.csv',index=False)

# If using Drive
#avg_df.to_csv('drive/MyDrive/CODE/Machine Learning/clustering/preprocessed_data/AvgPriceAndSalesVolume.csv',index=False)

---
Regression

In [ ]:
from sklearn.linear_model import LinearRegression

def regression(x,y):
    lr = LinearRegression()
    data = pd.DataFrame([x,y]).T
    data.columns = ['x','y']
    lr.fit(data[['x']],data['y'])

    return [lr.coef_[0],lr.intercept_]

regression([1,2,3,4,5],[11,11,11,22,22])

[3.3000000000000003, 5.5]

In [ ]:
gr_data = df.sort_values('DATE').groupby(['ASIN','MARKETPLACE','BRAND_NAME']).agg({"ORGANIC_UNITS_SOLD":[lambda x: list(x),lambda x: len(list(x))],
                                                                                "SALE_PRICE_EUR":[lambda x: list(x),lambda x: len(list(x))]}).reset_index()


gr_data.columns = ['ASIN','MARKETPLACE','BRAND_NAME','ORGANIC_UNITS_SOLD_list','ORGANIC_UNITS_SOLD_count',
                 'SALE_PRICE_EUR_list','SALE_PRICE_EUR_count']

gr_data_refined = gr_data[gr_data['ORGANIC_UNITS_SOLD_count'] > 2]
gr_data_refined['regression_vals'] = gr_data_refined.apply(lambda x: regression(x['ORGANIC_UNITS_SOLD_list'],x['SALE_PRICE_EUR_list']),axis=1)

subset = gr_data_refined[['ASIN','MARKETPLACE','BRAND_NAME','regression_vals']]
subset.to_csv('preprocessed_data/Regression_vals.csv',index=False)

# If using Drive
#subset.to_csv('drive/MyDrive/CODE/Machine Learning/clustering/preprocessed_data/Regression_vals.csv',index=False)